In [51]:
import requests

url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

response = requests.get(url)
if response.status_code == 200:
    print("下載成功")

下載成功


In [52]:
response.encoding

In [53]:

print(type(response.json()))
response.json()

<class 'dict'>


{'cwbopendata': {'@xmlns': 'urn:cwb:gov:tw:cwbcommon:0.1',
  'identifier': '659cdd47-a667-d74b-85cf-a0314a41ec97',
  'sender': 'weather@cwb.gov.tw',
  'sent': '2023-08-12T10:57:02+08:00',
  'status': 'Actual',
  'msgType': 'Issue',
  'source': 'MFC',
  'dataid': 'C0032-001',
  'scope': 'Public',
  'dataset': {'datasetInfo': {'datasetDescription': '三十六小時天氣預報',
    'issueTime': '2023-08-12T11:00:00+08:00',
    'update': '2023-08-12T10:57:02+08:00'},
   'location': [{'locationName': '臺北市',
     'weatherElement': [{'elementName': 'Wx',
       'time': [{'startTime': '2023-08-12T12:00:00+08:00',
         'endTime': '2023-08-12T18:00:00+08:00',
         'parameter': {'parameterName': '多雲午後短暫雷陣雨', 'parameterValue': '22'}},
        {'startTime': '2023-08-12T18:00:00+08:00',
         'endTime': '2023-08-13T06:00:00+08:00',
         'parameter': {'parameterName': '晴時多雲', 'parameterValue': '2'}},
        {'startTime': '2023-08-13T06:00:00+08:00',
         'endTime': '2023-08-13T18:00:00+08:00',
  

In [54]:
import requests
def download_data()->dict:
    url = 'https://opendata.cwb.gov.tw/fileapi/v1/opendataapi/F-C0032-001?Authorization=rdec-key-123-45678-011121314&format=JSON'

    response = requests.get(url)
    if response.status_code == 200:
        print("下載成功")
    return response.json()

In [55]:
def jsonDict_csvList(json)->list[dict]:
    '''
    - 傳入josn的資料結構
    - 取出需要的資料
    - 組合成list[dict]
    '''
    location = json['cwbopendata']['dataset']['location']
    weather_list = []
    for item in location:
        city_item = {}
        city_item['城市'] = item['locationName']
        city_item['啟始時間'] = item['weatherElement'][1]['time'][0]['startTime']
        city_item['結束時間'] = item['weatherElement'][1]['time'][0]['endTime']
        city_item['最高溫度'] = float(item['weatherElement'][1]['time'][0]['parameter']['parameterName'])
        city_item['最低溫度'] = float(item['weatherElement'][2]['time'][0]['parameter']['parameterName'])
        city_item['感覺'] = item['weatherElement'][3]['time'][0]['parameter']['parameterName']
        weather_list.append(city_item)
    return weather_list

In [56]:
def  get_fileName_path()->str:
    csvFileName = get_csvName()
    current_cwd = os.path.abspath(os.getcwd())
    abs_file_path = os.path.join(current_cwd,'data',csvFileName)
    return abs_file_path

In [57]:
import csv
def save_csv(data:list[dict],fileName) -> bool:
    '''
    - 將list[dict]儲存
    - 參數fileName要儲存的檔案名
    '''
    with open(fileName,mode='w',encoding='utf-8',newline='') as file:
        fieldnames = ['城市', '啟始時間','結束時間','最高溫度','最低溫度','感覺']
        writer = csv.DictWriter(file,fieldnames=fieldnames)
        writer.writeheader()
        writer.writerows(data)

In [58]:
json_data = download_data() #下載資料
csv_list = jsonDict_csvList(json_data) #解析json的資料成為csv的結構
save_csv(csv_list,"今天天氣.csv") #儲存資料 

下載成功


In [59]:
from datetime import datetime
import pytz #要換成自己的時區，不然會抓server所在地點的時間
def get_csvName()->str:
    tw_timezone = pytz.timezone('Asia/Taipei')
    current_time = datetime.now(tw_timezone)
    fileName = f"{current_time.year}年{current_time.month}月{current_time.day}日"
    return fileName

In [60]:
csvFileName = get_csvName()
csvFileName

'2023年8月12日'

In [61]:
#絕對路徑
import os
current_cwd = os.path.abspath(os.getcwd())
abs_file_path = os.path.join(current_cwd, 'data', csvFileName)
abs_file_path

'/workspaces/0701/data/2023年8月12日'

In [62]:
import os
def check_file_exist()->bool:
    csvFileName = get_csvName()
    current_cwd = os.path.abspath(os.getcwd())
    abs_file_path = os.path.join(current_cwd,'data',csvFileName)
    print(abs_file_path)
    if os.path.exists(abs_file_path):
        return True
    else:
        return False
    
check_file_exist()

/workspaces/0701/data/2023年8月12日


False

In [63]:
if check_file_exist():
    print('存在')
else:
    print('不存在')

/workspaces/0701/data/2023年8月12日
不存在


In [64]:
if not check_file_exist():
    print('不存在')
    json_data = download_data()
    csv_list = jsonDict_csvList(json_data)
    is_save = save_csv(csv_list, get_csvName())
    if is_save:
        print('存檔成功') 

/workspaces/0701/data/2023年8月12日
不存在


下載成功


In [67]:
import pandas as pd
file_path = get_fileName_path()
dataFrame = pd.read_csv(file_path)
dataFrame

FileNotFoundError: [Errno 2] No such file or directory: '/workspaces/0701/data/2023年8月12日'